# EE 304 - Neuromorphics: Brains in Silicon


## From Dimensionless Models to Silicon Circuits

So far we have figured out how to compute with networks of heterogeneuos spiking neurons

- We employed the Neural Engineering Framework's three principles:
    - **Representation**: Allows us to encode vector quantities in pools of spiking neurons and to decode these spike trains 
    - **Transformation**: Allows us to transform these vector quantities using weighted connections from one pool to another
    - **Dynamics**: Allows us to perform transformations that depend on what happened at an earlier time

### Silicon Circuit Implementations

<img src="files/lecture6/LPF-Response.png" width="360">

We will now start looking at how to implement heterogenous networks of spiking neurons in silicon, starting with design a circuit that implements the leaky membrane model. The figure show's its response to a current pulse.

- We will first implement a neuron's leaky membrane, realizing its exponential decay with MOS transistors
    - **Subthreshold Operation**: We operate the transistors below their threshold voltage, where the device is said to be "off" 
        - **Pro: Ultra-low power**: The transistor passes femtoamps to nanoamps, consuming very little power
        - **Con: Sensitive to mismatch**: This current incresese exponentially with the gate-voltage, making it extremely sensitive to threshold voltage mismatch

### Dimensionless Models

<img src="files/lecture6/rc_circuit.png" width="230">
- Remember that a neuron's leaky membrane may be modeled as:

$$C\frac{dV}{dt} = -G_{\rm leak}(V-E_{\rm leak})+I_{\rm in}$$

- Setting $\tau = C/G_{\rm leak}$, $v = (V-E_{\rm leak})/V_n$, and $u=I_{\rm in}/(G_{\rm leak}V_n)$ yields the dimensionless form:

$$\tau\frac{dv}{dt} = -v + u$$

- Here, $v$ is the voltage relative to $E_{\rm leak}$ in units of $V_n$, some convenient amount of voltage, and $u$ is the current in units of $G_{\rm leak}V_n$

- This is a generally applicable method for transforming dimensioned models into dimensionless ones 

### Leaky Membrane's Current-Mode Silicon Circuit 

<img src="files/lecture6/generic-lpf-ckt.png" width="360">

- We choose the following representations:
    - A current $I_v$ represents the dimensionless membrane voltage $v$ 
    - A current $I_u$ represents the dimensionless input current $u$

- We use the following circuit elements:
    - A capacitor models the membrane's capacitance
        - Its voltage ($V_v$) depends on the current fed to it as: $dV_v/dt = I_{\rm C}/C$
    - A subcircuit feeds a current $I_C$ to the capacitor
        - It computes $I_C$ as a function of $I_u$ and $I_v$ so as to implement the model
    - A transistor converts the capacitor's voltage ($V_v$) to $I_v$
        - $I_{\rm D}(V_v)$ defines the dependence of its drain current on its gate voltage

### How should $I_C$ depend on $I_u$ and $I_v$?

<img src="files/lecture6/generic-lpf-ckt.png" width="360">

- To figure this out, we first covert all currents to dimensionless form by defining a unit current $I_n$:
    - Thus, $u = I_u/I_n$ and $v = I_v/I_n$ 

- Substituting these expressions into the dimensionless model yields

    $$ \tau\frac{d}{dt}\left(\frac{I_v}{I_n}\right) = -\frac{I_v}{I_n} + \frac{I_u}{I_n} $$

    $$ \tau\frac{dI_v}{dt} = -I_v + I_u $$

    $$ \tau\frac{dI_v}{dV_v}\frac{dV_v}{dt} = -I_v + I_u $$

    $$ \tau\frac{dI_{\rm D}(V_v)}{dV_v}\frac{I_C}{C} = -I_v + I_u $$

- The derivative of a transistor's current with respect to its gate voltage, 
$dI_{\rm D}(V_{\rm G})/dV_{\rm G}$, is called its transconductance, $g_{\rm M}(V_{\rm G})$

- Substituting this in the previous expression yields 
    - $$ I_C = \frac{C/\tau}{g_{\rm M}(V_v)}(I_u - I_v) $$


### Subtheshold MOS Implementation:  $I_{\rm C}(I_u,I_v)=(I_u/I_v-1)I_{\rm lk}$ 

<img src="files/lecture6/intermediate-lpf-circuit.png" width="320">

- For the subtheshold MOS transistor in *saturation* ($V_{\rm DS}>4U_T$): 

    $$ I_{\rm D}(V_{\rm G}) = I_0 e^{(\kappa V_{\rm G}-V_{\rm S})/U_T} $$
 
    $$ g_{\rm M}(V_{\rm G}) \equiv \frac{dI_{\rm D}(V_{\rm G})}{dV_{\rm G}} 
                            = \frac{\kappa}{U_T}I_0 e^{(\kappa V_{\rm G}-V_{\rm S})/U_T} 
                            = \frac{\kappa I_{\rm D}}{U_T} $$

- Hence, we have 

\begin{align}
    I_C =& \frac{C/\tau}{\kappa I_{v}/U_T}(I_u - I_v)\\
        =& \frac{I_{\rm lk}}{I_{v}}(I_u - I_v)\\
        =& \frac{I_u}{I_v}I_{\rm lk} - I_{\rm lk}
\end{align}
- where $I_{\rm lk} = (C U_T)/(\kappa\tau)$

### Subtheshold MOS Implementation: Calculating $I_{\rm C}^+=(I_u/I_v)I_{\rm lk}$ 

<img src="files/lecture6/multiply-divide.png" width="640">

- As the subtheshold ($V_{\rm G}<V_{\rm th}$) MOS transistor's current is exponentially related to its gate and source voltages: 

    $$ I_{\rm D}(V_{\rm G},V_{\rm S}) = I_0 e^{(\kappa V_{\rm G}-V_{\rm S})/U_T} $$
    
    - Terminal voltages are measured relative to the bulk terminal ($V_{\rm B}$)


- By taking logarithms, we obtain: 
          
    $$ \ln(I_{\rm D}) = \ln(I_0)+(\kappa V_{\rm G}-V_{\rm S})/U_T $$
    
- If the transistor's source terminal is tied to its bulk terminal, we get $V_{\rm S}=0$, which yields the simpler expression
          
    $$ \ln(I_{\rm D}) = \ln(I_0) + \kappa V_{\rm G}/U_T $$

- We use this relationship to express $I_{\rm C}^+=(I_u/I_v)I_{\rm lk}$ in terms of the gate-bulk voltages $V_u$, $V_v$ and $V_{\rm lk}$ of transistors carrying the currents $I_u$, $I_v$, and $I_{\rm lk}$, respectively:
\begin{align}
    \ln(I_{\rm C}^+) &= \ln(I_u)+\ln(I_{\rm lk})-\ln{I_v}\\
                             &= (\ln(I_0) + \kappa V_u/U_T) + (\ln(I_0) + \kappa V_{\rm lk}/U_T) - (\ln(I_0) + \kappa V_v/U_T)
\end{align}

- Exponentiating both sides yields

    $$ I_{\rm C}^+ = I_0 e^{\kappa(V_u + V_{\rm lk} -V_v)/U_T} $$

- Implying that $I_{\rm C}^+$ can be obtained by setting the gate-bulk voltage of a transistor to $V_u + V_{\rm lk} -V_v$

- This may be arranged as shown in the figure
- The current $(I_u/I_v)I_{\rm lk}$ would flow from the transistor's source onto the capacitor

### Subtheshold MOS Implementation: Complete Circuit Design 

<img src="files/lecture6/complete-lpf.png" width="520">

- We have changed the $I_{\rm C}(I_u,I_v)$ subcircuit slightly to eliminate the battery: 
    - There is a negative feedback-loop around the first two transistors 
    - Given $I_u$ and $I_{\rm lk}$, it sets $V_u$ and $V_{\rm lk}$ appropriately,  

- This circuit satisfies the differential equation

\begin{split}
    \underbrace{ \frac{C U_T}{\kappa I_{\rm lk}} }_{\tau}\underbrace{ \frac{\dot{I}_v}{I_{\rm n}} }_{\dot{v}}\
    &= -\underbrace{ \frac{I_v}{I_{\rm n}} }_{v} + \underbrace{ \frac{I_u}{I_{\rm n}} }_{u}
\end{split}
 
- This mapping tells you how to set $I_{\rm lk}$ and $I_u$, given $\tau$ and $u$ specified by the user:

\begin{equation*}
  I_{\rm lk} = \frac{C U_T}{\kappa \tau} \;\; {\rm and} \;\; I_u = u I_n
\end{equation*}
